In [ ]:
%matplotlib nbagg

import numpy as np
import matplotlib.pyplot as plt

from copy import deepcopy
from math import sqrt, atan2, sin, cos
from matplotlib import animation
from matplotlib.collections import PathCollection

## Scientific Computing Project : Orbiting Bodies

#### Student   : Leonardo Bohac

This project will present a bidimensional gravitational system simulator. As for simplification, some assumptions will be made:

<br>

- The bodies are circle-shaped, and the force between two of them is given by Newton's Gravitational Law, taking them as mass-points.
<br>

- $\text{Radius}_\text{body} = \sqrt{\text{Mass}_\text{body}}$
<br>

- $G = 1$ (*Gravitational Constant*)


In addition to the usual laws, there will be a collision protocol which takes action when two (or more) bodies collide with one another ($D < R_1 + R_2$). When that happens, their masses will be added up onto a new body, also preserving their linear momentum. More than two bodies can collide in a transitive sense.

The code bellow defines the two types (classes) we'll be using, Body and Simulation:

In [ ]:
class Body:
    
    def __init__(self, m, x, y, vx, vy):
        
        self.m  = m
        self.x  = x
        self.y  = y
        self.vx = vx
        self.vy = vy
        
        self.r = sqrt(m)
        
        self.next_x = x
        self.next_y = y
        
        self.collision = None
        
    def DistanceTo(self, body):
        
        return sqrt((self.x - body.x)**2 + (self.y - body.y)**2)
    
    def ForceFrom(self, body, G=1):
        
        D = self.DistanceTo(body)
        
        if(D == 0): return (0,0)
        
        f = (G * self.m * body.m) / D**2
        
        delta_x  = body.x - self.x
        delta_y  = body.y - self.y
        
        agl = atan2(delta_y, delta_x)
        
        fx = f * cos(agl)
        fy = f * sin(agl)
        
        return (fx, fy)
    
    def ResultingForce(self, Simulation):
        
        Fx = 0
        Fy = 0
        
        for body in Simulation.Bodies:
            
            f = self.ForceFrom(body)
            
            Fx += f[0]
            Fy += f[1]
            
        return (Fx, Fy)
    
    def CollidedWith(self, body):
        
        return self.DistanceTo(body) < self.r + body.r

In [ ]:
class Simulation:
    
    def __init__(self, Bodies):
        
        self.Bodies = Bodies
        
        self.Evolution = [deepcopy(Bodies)]
        
        self.CollisionProtocol()
        
    def Update(self, dt):
        
        self.CollisionProtocol()
        
        for body in self.Bodies:
            
            body.next_x += body.vx * dt
            body.next_y += body.vy * dt

            F = body.ResultingForce(self)
            
            body.vx += F[0] / body.m
            body.vy += F[1] / body.m
            
        for body in self.Bodies:
            
            body.x = body.next_x
            body.y = body.next_y
        
        self.Evolution.append(deepcopy(self.Bodies))
        
    def Run(self, T, dt=2e-2):
        
        self.__init__(self.Evolution[0])
        
        u = int(T/dt)-1
        
        for _ in range(u):
            
            self.Update(dt)
        
    def CollisionProtocol(self):
        
        b = len(self.Bodies)
        
        Collisions = {}
        
        c = 0
        for i in range(b-1):
            
            for j in range(i+1, b):
                
                body_1 = self.Bodies[i]
                body_2 = self.Bodies[j]
                
                if(body_1.CollidedWith(body_2)):
                    
                    c_1 = body_1.collision
                    c_2 = body_2.collision
                    
                    if(c_1 == None and c_2 == None):
                        
                        body_1.collision = c
                        body_2.collision = c
                        
                        Collisions[c] = [body_1, body_2]
                        c += 1
                        continue

                    if(c_1 == None and c_2 != None):
                        
                        body_1.collision = body_2.collision
                        
                        Collisions[c_2].append(body_1)
                        continue
                        
                    if(c_1 != None and c_2 == None):
                        
                        body_2.collision = body_1.collision
                        
                        Collisions[c_1].append(body_2)
                        continue
                        
                    if(c_1 != None and c_2 != None and c_1 != c_2):
                        
                        for body in Collisions[c_2]:
                                
                            body.collision = c_1
                                
                        Collisions[c_1] += Collisions[c_2]
                        del Collisions[c_2]
    
        for c in Collisions:
            
            Collision = Collisions[c]
            
            M  = 0
            X  = 0
            Y  = 0
            Vx = 0
            Vy = 0
            
            for body in Collision:
                
                m  = body.m
                x  = body.x
                y  = body.y
                vx = body.vx
                vy = body.vy
                
                M  += m
                X  += m * x
                Y  += m * y
                Vx += m * vx
                Vy += m * vy
                
                self.Bodies.remove(body)
            
            X  /= M
            Y  /= M
            Vx /= M
            Vy /= M
            
            self.Bodies.append(Body(M, X, Y, Vx, Vy))
        
        for body in self.Bodies:
            body.collision = None
    
    
    def Animate(self):
        
        M = [[body.m for body in self.Evolution[i]] for i in range(len(self.Evolution))]
        X = [[body.x for body in self.Evolution[i]] for i in range(len(self.Evolution))]
        Y = [[body.y for body in self.Evolution[i]] for i in range(len(self.Evolution))]
        R = [[body.r for body in self.Evolution[i]] for i in range(len(self.Evolution))]
                
        Center_X = [sum([m*x for m,x in zip(M[i],X[i])]) / sum(M[i]) for i in range(len(M))]
        Center_Y = [sum([m*y for m,y in zip(M[i],Y[i])]) / sum(M[i]) for i in range(len(M))]
        
        X = [[x - Center_X[i] for x in X[i]] for i in range(len(X))]
        Y = [[y - Center_Y[i] for y in Y[i]] for i in range(len(Y))]
        
        x_max = max([max([X[i][j] + R[i][j] for j in range(len(M[i]))]) for i in range(len(M))])
        y_max = max([max([Y[i][j] + R[i][j] for j in range(len(M[i]))]) for i in range(len(M))])
        x_min = min([min([X[i][j] - R[i][j] for j in range(len(M[i]))]) for i in range(len(M))])
        y_min = min([min([Y[i][j] - R[i][j] for j in range(len(M[i]))]) for i in range(len(M))])
        
        spread = max([abs(x_max),abs(y_max),abs(x_min),abs(y_min)])*1.02
        
        X_Scat = [x for sub in X for x in sub]
        Y_Scat = [y for sub in Y for y in sub]
        R_Scat = [r for sub in R for r in sub]
        
        fig, ax = plt.subplots(figsize=(4,4))
        
        ax.set_aspect('equal');
        ax.set_xticks([])
        ax.set_yticks([])
        
        ax.set_xlim(-spread, spread)
        ax.set_ylim(-spread, spread)
        
        s_ratio = ax.get_position().transformed(fig.transFigure).width/1.4
        
        S_Scat = [((r/spread) * s_ratio)**2 for r in R_Scat]
        
        ax.scatter(X_Scat, Y_Scat, s=S_Scat)
        
        pc = ax.get_children()[0]
        assert(isinstance(pc, PathCollection))
        
        offsets = pc.get_offsets()
        for offset in offsets:
            offset.mask[0] = True
            offset.mask[1] = True
        
        Bpf = [len(m) for m in M]
        Bcs = list(np.cumsum(np.array(Bpf)))
        
        frames = len(M)
        def animate(i):
            
            bpf = Bpf[i-1]
            bcs = Bcs[i-1]
            
            if(i > 0):
                
                for j in range(bpf):

                    offsets[bcs+j].mask[0] = False
                    offsets[bcs+j].mask[1] = False

                lbpf = Bpf[i-2]

                for j in range(1, lbpf+1):

                    offsets[bcs-j].mask[0] = True
                    offsets[bcs-j].mask[1] = True
        
            if(i == frames-1):
                
                for j in range(bpf):

                    offsets[bcs+j].mask[0] = True
                    offsets[bcs+j].mask[1] = True
        
        self.anim = animation.FuncAnimation(fig, animate, frames=frames, interval=5)
        #self.anim.save('../orbit.gif', writer='pillow', fps=50) # change interval above to 20 if saving

We can now run a few simulations and visualize them:

In [ ]:
Bodies = [Body(1,0,0,0,0), Body(.4,3,0,0,5)]

S = Simulation(Bodies)
S.Run(30)

S.Animate()

In [ ]:
Bodies = [Body(1e3,0,0,0,0), Body(1e1,50,0,0,30), Body(1e0,-60,0,0,32), Body(1e-1,72,0,0,30)]

S = Simulation(Bodies)
S.Run(30)

S.Animate()

And see some collisions taking place:

In [ ]:
Bodies = [Body(1e3, 0,0,0, 0), 
          Body(1e1,50,0,0,40), Body(1e1,-50,0,0,40),
          Body(1e1,65,0,0,40), Body(1e1,-65,0,0,40)]

S = Simulation(Bodies)
S.Run(20)

S.Animate()

The animations were set so that each frame is centered at its center of mass.

______________